In [3]:
from typing import *
from pathlib import Path
import numpy as np
import lightgbm as lgb
# import optuna.integration.lightgbm as lgb2
import pandas as pd
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

root = Path("/data/natsuki/dataset_atmaCup11/checkpoints")
names = ["atma11simple_j4e5nofreeze", "atma11sortingdate_j4e5nofreeze", "atma11onehot_j4e5nofreeze", "atma11materialstechniques_j4e5nofreeze"]
epoch = 5


In [11]:
for name in names:
    for fold in range(3):
        test_features_df = pd.read_csv(root/f"{name}_3fold{fold}"/f"epoch{epoch}_3fold{fold}_test_features.csv")
        break
    break

In [9]:
df

,object_id,atma11simple_j4e5nofreeze_3fold0_e5_0
0,007c091616828798b5e1,1.534457
1,007f5e3620b458d77212,0.455873
2,00db4ef4b89904547a77,0.994816
3,0108b38da16211245fcd,1.892395
4,0110115b8b6036d9ab3c,1.764503
...,...,...
1308,ff327cfcf624d261e898,0.224658
1309,ff7e4ea4683cedabb534,1.619340
1310,ff9ed0536b98b67cee64,2.439260
1311,ffd794b7b311b7b7fd92,2.901760
